In [0]:
df=spark.read.format("csv").option("header",True).option("inferSchema",True).load("abfss://raw@dbstorageprac.dfs.core.windows.net/Customer.csv")

In [0]:
df.display()

In [0]:
df = df.toDF(*[col.replace(' ', '_') for col in df.columns])
df.write.format("delta").mode("overwrite").saveAsTable("dbcatalog.bronze.customer")


In [0]:
%sql
select * from dbcatalog.bronze.customer

In [0]:
%sql
ALTER TABLE dbcatalog.bronze.customer SET TAGS ('class.email_address' = 'confidential');


In [0]:
%sql
CREATE EXTERNAL TABLE IF NOT EXISTS dbcatalog.bronze.orders
USING DELTA
LOCATION 'abfss://bronze@dbstorageprac.dfs.core.windows.net/orders';


In [0]:
from pyspark.sql.functions import input_file_name
df=(spark.readStream.format("cloudFiles").option("cloudFiles.format","json").option("cloudFiles.schemaLocation","abfss://myexternal@dbstorageprac.dfs.core.windows.net/schema_location").option("cloudFiles.multiLine", "true").load("abfss://raw@dbstorageprac.dfs.core.windows.net/"))

In [0]:
display(df)

In [0]:
spark.read.format("json").option("multiline",True).load("abfss://raw@dbstorageprac.dfs.core.windows.net/Orders.json")

In [0]:
(
  df.writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://myexternal@dbstorageprac.dfs.core.windows.net/checkpoints/autoloader")
    .outputMode("append")
    .start("abfss://bronze@dbstorageprac.dfs.core.windows.net/your_custom_path")
)
